<h1> DS200A Computer Vision Assignment</h1>

<h2>  Part Four: Extension Activities</h2>	


TensorFlow (Optional)- Now, try using TensorFlow to categorize your images. The accuracy should be significantly higher due to the usage of nueral nets



In [1]:
#Dataset processing
import os, glob
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import cv2
import time

def load_data(train_path, img_w, img_h):
    print('Reading training images')
    img_list, label_list = [], []
    classes = os.listdir(train_path)
    for index, fld in enumerate(classes):
        path = os.path.join(train_path, fld, '*g')
        files = glob.glob(path)
        for fl in files:
            image = cv2.imread(fl)
            image = cv2.resize(image, (img_w, img_h), cv2.INTER_LINEAR)
            img_list.append(image), label_list.append(index)
    img_list, label_list = np.array(img_list, dtype=np.uint8), np.array( label_list, dtype=np.int32 )
    img_list = img_list.astype('float32')
    img_list = img_list / 255
    return img_list, label_list

img_w, img_h = 256, 256
train_dir, test_dir = "20_categories_training/", "20_Validation/"

data, label = load_data(train_dir, img_w, img_h)
print(data.shape, label.shape)
idx = np.arange( data.shape[0] )
np.random.shuffle(idx)
data, label = data[idx], label[idx]

split_ratio = 0.9
split = np.int(data.shape[0]* split_ratio)
x_train, y_train = data[:split], label[:split]
x_val, y_val =data[split:], label[split:]

/Users/sheng/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Reading training images
(1501, 256, 256, 3) (1501,)


In [2]:
#tensorflow model
def inference(input_tensor, batch_size, n_classes, train):
    # conv1
    with tf.variable_scope("layer1-conv1") as scope:
        weights = tf.get_variable("weights", shape=[3, 3, 3, 16], dtype=tf.float32, initializer=tf.truncated_normal_initializer(stddev=0.1, dtype=tf.float32))
        biases = tf.get_variable("biases", shape=[16], dtype=tf.float32, initializer=tf.constant_initializer(0.1))
        conv = tf.nn.conv2d(input_tensor, weights, strides=[1, 1, 1, 1], padding="SAME")
        activation = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(activation, name="conv1")

    # pool1 && norm1
    with tf.variable_scope("layer2-pooling_lrn_1") as scope:
        pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="SAME", name="pooling1")
        norm1 = tf.nn.lrn(pool1, depth_radius=4, bias=1.0, alpha=0.001/9.0, beta=0.75, name='layer_norm1')

    # conv2
    with tf.variable_scope("layer3-conv2") as scope:
        weights = tf.get_variable("weights", shape=[3, 3, 16, 16], dtype=tf.float32, initializer=tf.truncated_normal_initializer(stddev=0.1, dtype=tf.float32))
        biases = tf.get_variable("biases", shape=[16], dtype=tf.float32, initializer=tf.constant_initializer(0.1))
        conv = tf.nn.conv2d(norm1, weights, strides=[1, 1, 1, 1], padding="SAME")
        activation = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(activation, name="conv2")

    # pool2 && norm2
    with tf.variable_scope("layer4-pooling2_lrn") as scope:
        pool2 = tf.nn.max_pool(conv2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding="SAME", name="pooling2")
        norm2 = tf.nn.lrn(pool2, depth_radius=4, bias=1.0, alpha=0.001/9.0, beta=0.75, name='layer_norm2')
#         print(norm2.shape)
    # full-connect1
    with tf.variable_scope("layer5-fc1") as scope:
        reshape = tf.reshape(norm2, shape=[batch_size, -1])
        dim = reshape.get_shape()[1].value
        weights = tf.get_variable("weights", shape=[65536, 128], dtype=tf.float32, initializer=tf.truncated_normal_initializer(stddev=0.005, dtype=tf.float32))
        biases = tf.get_variable("biases", shape=[128], dtype=tf.float32, initializer=tf.constant_initializer(0.1))
        fc1 = tf.nn.relu(tf.matmul(reshape, weights) + biases, name="fc1")
        if train: fc1 = tf.nn.dropout(fc1, 0.5)
    # full_connect2
    with tf.variable_scope("layer6-fc2") as scope:
        weights = tf.get_variable("weights", shape=[128, 128], dtype=tf.float32, initializer=tf.truncated_normal_initializer(stddev=0.005, dtype=tf.float32))
        biases = tf.get_variable("biases", shape=[128], dtype=tf.float32, initializer=tf.constant_initializer(0.1))
        fc2 = tf.nn.relu(tf.matmul(fc1, weights) + biases, name="fc2")
        if train: fc2 = tf.nn.dropout(fc2, 0.5)

    # softmax
    with tf.variable_scope("layer7-fc3") as scope:
        weights = tf.get_variable("weights", shape=[128, n_classes], dtype=tf.float32, initializer=tf.truncated_normal_initializer(stddev=0.005, dtype=tf.float32))
        biases = tf.get_variable("biases", shape=[n_classes], dtype=tf.float32, initializer=tf.constant_initializer(0.1))
        logit = tf.add(tf.matmul(fc2, weights), biases, name="fc3")
    return logit

channel, num_classes = 3, 20
model_path = "model/model.ckpt"
batch_size = 64
lr = 0.001
x = tf.placeholder(tf.float32, shape=[None, img_w, img_h, channel], name='x')
labels = tf.placeholder(tf.int32, shape=[None,], name='label')

# regularizer = tf.contrib.layers.l2_regularizer(0.0001)
logits = inference(x, batch_size=batch_size, n_classes=num_classes, train=True)

b = tf.constant(value=1,dtype=tf.float32)
logits_eval = tf.multiply(logits,b,name='logits_eval') 

loss = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels)
train_op = tf.train.AdamOptimizer(learning_rate=lr).minimize(loss)
gold = tf.equal(tf.cast(tf.argmax(logits, 1), tf.int32), labels)    
acc = tf.reduce_mean(tf.cast(gold, tf.float32))

def minibatches(inputs=None, targets=None, batch_size=None, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batch_size + 1, batch_size):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batch_size]
        else:
            excerpt = slice(start_idx, start_idx + batch_size)
        yield inputs[excerpt], targets[excerpt]

In [3]:
n_epoch = 10                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
saver = tf.train.Saver()
sess = tf.Session()  
sess.run(tf.global_variables_initializer())
for epoch in range(n_epoch):
    start_time = time.time()
    #training
    train_loss, train_acc, n_batch = 0, 0, 0
    for x_train_a, y_train_a in minibatches(x_train, y_train, batch_size, shuffle=True):
        _,err,ac=sess.run([train_op,loss,acc], feed_dict={x: x_train_a, labels: y_train_a})
        train_loss += err; train_acc += ac; n_batch += 1
    print("------------%d------------" % epoch)
    print("   train loss: %f" % (np.sum(train_loss)/ n_batch))
    print("   train acc: %f" % ( np.sum(train_acc)/ n_batch))

    #validation
    val_loss, val_acc, n_batch = 0, 0, 0
    for x_val_a, y_val_a in minibatches(x_val, y_val, batch_size, shuffle=False):
        err, ac = sess.run([loss,acc], feed_dict={x: x_val_a, labels: y_val_a})
        val_loss += err; val_acc += ac; n_batch += 1
    print("   validation loss: %f" % (np.sum(val_loss)/ n_batch))
    print("   validation acc: %f" % (np.sum(val_acc)/ n_batch))
saver.save(sess, model_path)
sess.close()

   train loss: 190.646124
   train acc: 0.078869
   validation loss: 190.196976
   validation acc: 0.085938
   train loss: 186.449254
   train acc: 0.114583
   validation loss: 177.802826
   validation acc: 0.156250
   train loss: 175.131278
   train acc: 0.170387
   validation loss: 164.359985
   validation acc: 0.179688
   train loss: 168.182850
   train acc: 0.204613
   validation loss: 157.688431
   validation acc: 0.234375
   train loss: 160.565034
   train acc: 0.248512
   validation loss: 153.880447
   validation acc: 0.265625
   train loss: 154.476981
   train acc: 0.265625
   validation loss: 149.247223
   validation acc: 0.289062
   train loss: 149.821359
   train acc: 0.281994
   validation loss: 142.804626
   validation acc: 0.289062
   train loss: 146.849144
   train acc: 0.290923
   validation loss: 140.171204
   validation acc: 0.320312
   train loss: 143.176165
   train acc: 0.309524
   validation loss: 146.228104
   validation acc: 0.273438
   train loss: 138.795015
  

In [4]:
def load_test(train_path, test_path, img_w, img_h):
    print('Reading testing images')
    img_list, label_list = [], []
    classes = os.listdir(train_path)
    files = glob.glob(test_path)
    for fl in files:
        image = cv2.imread(fl)
        image = cv2.resize(image, (img_w, img_h), cv2.INTER_LINEAR)
    img_list = np.array(img_list, dtype=np.uint8)
    img_list = img_list.astype('float32')
    img_list = img_list / 255
    return img_list, classes

test_data = load_test(train_path, test_path, img_w, img_h)
with tf.Session() as sess:
    saver = tf.train.import_meta_graph('model/model.ckpt.meta')
    saver.restore(sess,tf.train.latest_checkpoint('model/'))
    graph = tf.get_default_graph()
    x = graph.get_tensor_by_name("x:0")
    feed_dict = {x:test_data}
    logits = graph.get_tensor_by_name("logits_eval:0")
    classification_result = sess.run(logits,feed_dict)
#     print(tf.argmax(classification_result,1).eval())
    output = tf.argmax(classification_result, 1).eval()

OSError: File model/model.ckpt.meta does not exist.

In [ ]:
def TF():
#     raise NotImplemented()
    